<a href="https://colab.research.google.com/github/NickSlm/ml1/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [2]:
dataset = keras.datasets.imdb.load_data()

In [3]:
(X_train, y_train), (X_test, y_test) = dataset

In [4]:
test_seq = X_train[0][:10]

In [5]:
word_index = keras.datasets.imdb.get_word_index()

In [6]:
id_to_word = dict((id_ + 3,word) for word, id_ in word_index.items())
id_to_word [1] = "[START]"
id_to_word [2] = "[OOV]"
" ".join(id_to_word[id_] for id_ in test_seq)

'[START] this film was just brilliant casting location scenery story'

In [7]:
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [8]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [9]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")    # replace all <br>, <br/>
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [10]:
from collections import Counter

In [11]:
vocab = Counter()
for X_batch, y_batch in dataset["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocab.update(list(review.numpy()))

In [12]:
n_words = 10000
truncated_vocab = [word for word, times in vocab.most_common()[:n_words]]

In [13]:
word_ids = tf.range(len(truncated_vocab), dtype=tf.int64)
words = tf.constant(truncated_vocab)
table_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(table_init, oov_buckets)

In [14]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [15]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch

train_set = dataset["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [16]:
model = keras.models.Sequential([
    keras.layers.Embedding(n_words + oov_buckets, 128, mask_zero = True, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

Reusing Pretrained Embedding

In [17]:
import tensorflow_hub as hub

In [19]:
model = keras.models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)

In [21]:
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 13s 14ms/step - loss: 0.5472 - accuracy: 0.7266
Epoch 2/5
782/782 [==============================] - 8s 10ms/step - loss: 0.5145 - accuracy: 0.7466
Epoch 3/5
782/782 [==============================] - 5s 6ms/step - loss: 0.5093 - accuracy: 0.7508
Epoch 4/5
782/782 [==============================] - 5s 6ms/step - loss: 0.5057 - accuracy: 0.7533
Epoch 5/5
782/782 [==============================] - 6s 7ms/step - loss: 0.5027 - accuracy: 0.7564
